In [14]:
# CURL query: http://web.archive.org/cdx/search/cdx?url=https://housing.berkeley.edu/rates-contracts-policies/rates/&output=json

In [9]:
import json

with open("housing_site_snapshots.json", "r") as f:
    data = json.load(f)

# Skip the header row and construct the archive URLs
urls = [
    f"http://web.archive.org/web/{entry[1]}/{entry[2]}"
    for entry in data[1:]
]

In [18]:
urls

['http://web.archive.org/web/20211222053825/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20220127224043/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20220303194920/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20220405102602/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20220510230410/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20220522230535/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20220625035214/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20220721100123/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20221003102631/https://housing.berkeley.edu/rates-contracts-policies/rates/',
 'http://web.archive.org/web/20230202

In [19]:
from datetime import datetime

def convert_timestamp(timestamp):
    """Convert a timestamp in 'YYYYMMDDHHMMSS' format to a readable string."""
    try:
        readable_format = datetime.strptime(timestamp, "%Y%m%d%H%M%S").strftime("%Y-%m-%d")
        return readable_format
    except ValueError:
        return "Invalid timestamp format. Please use 'YYYYMMDDHHMMSS'."

# Example usage
timestamp = "20240811231741"
print(f"Readable Timestamp: {convert_timestamp(timestamp)}")

Readable Timestamp: 2024-08-11


In [20]:
timestamps = [convert_timestamp(entry[1]) for entry in data[1:]]
timestamps

['2021-12-22',
 '2022-01-27',
 '2022-03-03',
 '2022-04-05',
 '2022-05-10',
 '2022-05-22',
 '2022-06-25',
 '2022-07-21',
 '2022-10-03',
 '2023-02-02',
 '2023-06-07',
 '2023-11-24',
 '2023-12-02',
 '2024-02-09',
 '2024-04-20',
 '2024-07-13',
 '2024-07-21',
 '2024-08-01',
 '2024-08-07',
 '2024-08-08',
 '2024-08-08',
 '2024-08-09',
 '2024-08-11',
 '2024-08-12',
 '2024-08-13',
 '2024-08-20',
 '2024-08-22']

In [28]:
import pandas as pd
import os
import time

# Create a directory to save tables
if not os.path.exists("scraped_tables/dorm_housing"):
    os.makedirs("scraped_tables/dorm_housing")
if not os.path.exists("scraped_tables/campus_apartment_housing"):
    os.makedirs("scraped_tables/campus_apartment_housing")

url = urls[0]
url = url.strip()  # Remove extra whitespace
    
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML content
soup = BeautifulSoup(response.content, 'lxml')

# Find all tables in the HTML
tables = soup.find_all('table')

on_campus_housing_ids = ["Standard Room Projected Rates", 
                  "Premium Room Projected Rates (Blackwell)", 
                  "Room in Mini Suite Projected Rates", 
                  "Room in Suite Projected Rates", 
                  "Premium Room Projected Rates (Over Winter Break)"]

# Rate limit settings
RATE_LIMIT = 2  # Time in seconds between requests

# Loop through each URL
for n, url in enumerate(urls):
    url = url.strip()  # Remove extra whitespace
    
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'lxml')

    # Find all tables in the HTML
    tables = soup.find_all('table')

    # Process each table
    for i, table in enumerate(tables):
        df = pd.read_html(str(table))[0]  # Convert the table to a DataFrame
        
        # On campus housing
        if i < 5:
            filename = f"scraped_tables/dorm_housing/{on_campus_housing_ids[i]}_{timestamps[n]}.csv"
        else:
            filename = f"scraped_tables/campus_apartment_housing/table_{i}_{timestamps[n]}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved table to {filename}")
    
    # Rate limiting to avoid getting flagged
    if n < len(urls) - 1:  # Avoid unnecessary delay after the last request
        print(f"Waiting {RATE_LIMIT} seconds before the next request...")
        time.sleep(RATE_LIMIT)

Saved table to scraped_tables/dorm_housing/Standard Room Projected Rates_2024-08-07.csv
Saved table to scraped_tables/dorm_housing/Premium Room Projected Rates (Blackwell)_2024-08-07.csv
Saved table to scraped_tables/dorm_housing/Room in Mini Suite Projected Rates_2024-08-07.csv
Saved table to scraped_tables/dorm_housing/Room in Suite Projected Rates_2024-08-07.csv
Saved table to scraped_tables/dorm_housing/Premium Room Projected Rates (Over Winter Break)_2024-08-07.csv
Saved table to scraped_tables/campus_apartment_housing/table_5_2024-08-07.csv
Saved table to scraped_tables/campus_apartment_housing/table_6_2024-08-07.csv
Saved table to scraped_tables/campus_apartment_housing/table_7_2024-08-07.csv
Saved table to scraped_tables/campus_apartment_housing/table_8_2024-08-07.csv
Saved table to scraped_tables/campus_apartment_housing/table_9_2024-08-07.csv
Saved table to scraped_tables/campus_apartment_housing/table_10_2024-08-07.csv
Saved table to scraped_tables/campus_apartment_housing/t